<a href="https://colab.research.google.com/github/ProtossDragoon/paper_implementation_and_testing_tf2/blob/main/utils/Generate_TFRecord.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generate TFRecord

## Author

name : Janghoo Lee <br>
github : https://github.com/ProtossDragoon <br>
contact : dlwkdgn1@naver.com <br>
circle : https://github.com/sju-coml <br>
organization : https://web.deering.co/ <br>
published date : June, 2021


## Related Notebook

[Notebooks](https://github.com/ProtossDragoon/paper_implementation_and_testing_tf2/tree/main/notebooks)


# Environment

In [2]:
import tensorflow as tf
print(tf.__version__)

2.5.0


In [3]:
import os, sys
from tqdm.notebook import tqdm as tqdm

## Google Drive / Git

In [7]:
#!rm -r /content/gdrive/
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [38]:
HOME_DIR = "/content/gdrive/MyDrive"
WS = os.path.join(HOME_DIR, "ColabWorkspace")
DATA_DIR = os.path.join(HOME_DIR, 'data')
DATA_INDEX_DIR = os.path.join(DATA_DIR, 'index')
DATA_TFRECORD_DIR = os.path.join(DATA_DIR, 'tfrecord')

GIT_REPO_NAME = 'paper_implementation_and_testing_tf2'
GIT_WS = os.path.join(WS, GIT_REPO_NAME)

GIT_BRANCH = 'master'
GIT_USERNAME = None # Add herea
GIT_EMAIL = None
GIT_PASSWORD = None # Add here

In [9]:
!mkdir -p {WS}
%cd {WS}
!git clone https://github.com/ProtossDragoon/{GIT_REPO_NAME}.git
%cd {GIT_WS}
!git config --global user.name {GIT_USERNAME}
!git config --global user.email {GIT_EMAIL}
!git checkout -b {GIT_BRANCH}
!git checkout {GIT_BRANCH}
!git pull origin {GIT_BRANCH}
!git push origin {GIT_BRANCH}
!git branch
!git branch --set-upstream-to origin/{GIT_BRANCH}

/content/gdrive/MyDrive/ColabWorkspace
fatal: destination path 'paper_implementation_and_testing_tf2' already exists and is not an empty directory.
/content/gdrive/MyDrive/ColabWorkspace/paper_implementation_and_testing_tf2
fatal: A branch named 'master' already exists.
Switched to branch 'master'
fatal: Couldn't find remote ref master
fatal: could not read Username for 'https://github.com': No such device or address
  list
  main
* master
  resnet
  tftrt
  unet
error: the requested upstream branch 'origin/master' does not exist
hint: 
hint: If you are planning on basing your work on an upstream
hint: branch that already exists at the remote, you may need to
hint: run "git fetch" to retrieve it.
hint: 
hint: If you are planning to push out a new local branch that
hint: will track its remote counterpart, you may want to use
hint: "git push -u" to set the upstream config as you push.


# Convert

## Hyper Parameters

In [41]:
DATASETNAME = "aihubpedestrian" #@param {type:"string"}
TXT_FILE_DIR = os.path.join(DATA_INDEX_DIR, DATASETNAME.upper())
TXT_FILE_NAME = 'train' #@param ['train', 'val', 'test']
TXT_FILE_NAME += '.txt'
TXT_FILE_FULLPATH = os.path.join(TXT_FILE_DIR, TXT_FILE_NAME)

print("Indexfile generation result save '{}' at {}".format(TXT_FILE_NAME, TXT_FILE_FULLPATH))

Indexfile generation result save 'train.txt' at /content/gdrive/MyDrive/data/index/AIHUBPEDESTRIAN/train.txt


In [34]:
DATASETNAME = "aihubpedestrian" #@param {type:"string"}
TFRECORD_FILE_DIR = os.path.join(DATA_TFRECORD_DIR, DATASETNAME.upper())
TFRECORD_FILE_NAME = 'train' #@param ['train', 'val', 'test']
TFRECORD_FILE_NAME += '.tfrecord'
TFRECORD_FILE_FULLPATH = os.path.join(TFRECORD_FILE_DIR, TFRECORD_FILE_NAME)

print("TFRecord Conversion result save '{}' at {}".format(TFRECORD_FILE_NAME, TFRECORD_FILE_FULLPATH))

TFRecord Conversion result save 'train.tfrecord' at /content/gdrive/MyDrive/data/tfrecord/AIHUBPEDESTRIAN/train.tfrecord


## Generate Index file

# Debug

In [14]:
DATASET_DIR_CAMVID = os.path.join(DATA_DIR, 'camvid')
DATASET_TRAIN_X_DIR_CAMVID = os.path.join(DATASET_DIR_CAMVID, 'train')
DATASET_TRAIN_Y_DIR_CAMVID = os.path.join(DATASET_DIR_CAMVID, 'trainannot')
DATASET_VAL_X_DIR_CAMVID = os.path.join(DATASET_DIR_CAMVID, 'val')
DATASET_VAL_Y_DIR_CAMVID = os.path.join(DATASET_DIR_CAMVID, 'valannot')
DATASET_TEST_X_DIR_CAMVID = os.path.join(DATASET_DIR_CAMVID, 'test')
DATASET_TEST_Y_DIR_CAMVID = os.path.join(DATASET_DIR_CAMVID, 'testannot')

%cd {WS}
%ls -al

/content/gdrive/MyDrive/ColabWorkspace
total 20
drwx------  2 root root 4096 Jun 22 02:12 albumentations/
drwx------  2 root root 4096 Jun 25 02:12 {BACKUP_DIR}/
drwx------  2 root root 4096 Jun 22 02:12 efficientnet/
drwx------ 14 root root 4096 May 25 05:07 paper_implementation_and_testing_tf2/
drwx------  2 root root 4096 Jun 22 02:12 segmentation_models/


In [15]:
DATASET_DIR_PEDESTRIAN = os.path.join(DATA_DIR, 'aihubsidewalk')
DATASET_TRAIN_TXT_PEDESTRIAN = os.path.join(DATASET_DIR_PEDESTRIAN, 'train.txt')
DATASET_VALID_TXT_PEDESTRIAN = os.path.join(DATASET_DIR_PEDESTRIAN, 'val.txt')
DATASET_TEST_TXT_PEDESTRIAN = os.path.join(DATASET_DIR_PEDESTRIAN, 'test.txt')

%cd {WS}
%ls -al

/content/gdrive/MyDrive/ColabWorkspace
total 20
drwx------  2 root root 4096 Jun 22 02:12 albumentations/
drwx------  2 root root 4096 Jun 25 02:12 {BACKUP_DIR}/
drwx------  2 root root 4096 Jun 22 02:12 efficientnet/
drwx------ 14 root root 4096 May 25 05:07 paper_implementation_and_testing_tf2/
drwx------  2 root root 4096 Jun 22 02:12 segmentation_models/


In [16]:
# classes for data loading and preprocessing
import abc
class SegmentationDataset(abc.ABC):
    """Segmentation Dataset. Read images, apply augmentation and preprocessing transformations.
       Inheritance this class and implement fill_data method.
       - Inside of fill_data method, self.images_fps and self.mask_fps should be filled.
       - If you have RGB colorized masks, put {(int)classid: <tuple>(r,g,b)} shaped dictionary at self.class_id_to_rgb.
       
       
       Args:
            class_ids (list): values of classes ids       
            augmentation (albumentations.Compose): data transfromation pipeline 
                (e.g. flip, scale, etc.)
            preprocessing (albumentations.Compose): data preprocessing 
                (e.g. noralization, shape manipulation, etc.)
    """
    
    def __init__(self, class_ids=None, augmentation=None, preprocessing=None, resizing=None):

        # self.images_fps = user implementation 
        # self.masks_fps = user implementation        
        # self.class_id_to_rgb = user implementation
        self.class_ids = class_ids
        self.interested_class_id_rgb = None

        # func*
        self.augmentation = augmentation
        self.preprocessing = preprocessing
        self.resizing = resizing

        print('augmentation : {} -> {}'.format(type(augmentation), type(self.augmentation)))
        print('preprocessing : {} -> {}'.format(type(preprocessing), type(self.preprocessing)))
        print('resizing : {} -> {}'.format(type(resizing), type(self.resizing)))

        # mode
        self.mode = 'tf.keras.Sequence' # or tf.data.Dataset


    @abc.abstractmethod
    def fill_data(self):
        """Implement this mehtod and fill these member elements

        Member to elements:
            image_fps (str): file path to images
            masks_fps (str): file path to segmentation masks
        """
        pass


    def _readmask(self, i, class_id_to_rgb=None):
        """extract certain classes from mask (e.g. cars)
        """

        if hasattr(self, 'class_id_to_rgb') and (class_id_to_rgb is None):
            # If overrided class implemented self.class_id_to_rgb
            class_id_to_rgb = self.class_id_to_rgb

        if class_id_to_rgb is None:
            mask = cv2.imread(self.masks_fps[i], cv2.IMREAD_GRAYSCALE)
            masks = [(mask == v) for v in self.class_ids]
        else:
            self.interested_class_id_rgb = {}
            mask = cv2.imread(self.masks_fps[i])
            mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
            for k, v in class_id_to_rgb.items():
                if k in self.class_ids:
                    self.interested_class_id_rgb[k] = v

            ## debug
            # print(self.masks_fps[i]) ## debug
            # print(self.images_fps[i]) ## debug
            # print(image.shape, mask.shape) ## debug

            masks = []
            for class_id, colors in self.interested_class_id_rgb.items():
                # e.g. one class has two color {2: [(255, 128, 0), (255, 64, 0)]}
                if len(np.array(colors).shape) == 2:
                    temp_mask = []
                    for color in np.array(colors):
                        temp_mask.append(np.all(mask == color, axis=-1))    
                    masks.append(np.logical_or.reduce(temp_mask))
                else:
                    ## debug
                    # print('---\n', colors) ## debug
                    # print(np.all(mask == colors, axis=-1)) ## debug
                    # print('---') ## debug
                    # plt.figure() ## debug
                    # plt.imshow(np.all(mask == colors, axis=-1)) ## debug
                    ## debug
                    masks.append(np.all(mask == colors, axis=-1))

        mask = np.stack(masks, axis=-1).astype('int')

        ## debug
        # print('mask max : {}, min : {}, type : {}'.format(mask.max(), mask.min(), mask.dtype)) ## debug

        # add background if mask is not binary
        if mask.shape[-1] != 1:
            background = 1 - mask.sum(axis=-1, keepdims=True)
            mask = np.concatenate((mask, background), axis=-1)

        ## debug
        # print('mask max : {}, min : {}, type : {}'.format(mask.max(), mask.min(), mask.dtype)) ## debug

        return mask.astype('float')


    def _preprocessing_tf(self, image, mask):
        ret = self.preprocessing(image=image, mask=mask)
        image = ret['image']
        mask = ret['mask']
        return image, mask


    @tf.function(input_signature=(tf.TensorSpec(shape=[None, None, None], dtype=tf.uint8), 
                                  tf.TensorSpec(shape=[None, None, None], dtype=tf.uint8),
                                  tf.TensorSpec(shape=[], dtype=tf.bool),))
    def _preprocessing_fn(self, image, mask, preprocessing_exist):
        #tf.print('self._preprocessing_exist :', self._preprocessing_exist)
        if preprocessing_exist: # branch type : <image> tf.uint8 -> tf.float32
            #debugging code
            #tf.print('preprocessing_fn / main branch / <image> val :', tf.math.reduce_min(image), '~', tf.math.reduce_max(image))
            image = tf.cast(image, tf.float32)
            #debugging code
            #tf.print('preprocessing_fn / main branch / casting / <image> val :', tf.math.reduce_min(image), '~', tf.math.reduce_max(image))
            image, mask = tf.numpy_function(func=self._preprocessing_tf, inp=[image, mask], Tout=[tf.float32, tf.uint8])
            #debugging code
            #tf.print('preprocessing_fn / main branch / casting / preprocessing / <image> val :', tf.math.reduce_min(image), '~', tf.math.reduce_max(image))
            #tf.print('preprocessing_fn / main branch / casting / preprocessing / <image> tensorspec :', tf.TensorSpec.from_tensor(image))
        else: # branch type : <image> tf.uint8 -> tf.float32
            image = tf.cast(image, tf.float32)
            #debugging code
            #tf.print('preprocessing_fn / else branch / <image> val :',tf.math.reduce_min(image), '~', tf.math.reduce_max(image))
            #tf.print('preprocessing_fn / else branch / <image> tensorspec :', tf.TensorSpec.from_tensor(image))

        #debugging code
        #tf.print('preprocessing_fn / <image> val :', tf.math.reduce_min(image), '~', tf.math.reduce_max(image))
        #tf.print('preprocessing_fn / <image> tensorspec :', tf.TensorSpec.from_tensor(image))
        
        #categorical crossentropy loss function etc... wants float tensor, not uint8 tensor.
        # <mask> tf.uint8 -> tf.uint8
        mask = tf.cast(mask, tf.float32)
        
        return image, mask


    def _resizing_tf(self, image, mask):
        #debugging code
        #print(image, mask)
        #print(type(image), type(mask))
        #tf.print(type(image), type(mask))

        ret = self.resizing(image=image, mask=mask)
        return ret


    @tf.function(input_signature=(tf.TensorSpec(shape=[None, None, None], dtype=tf.uint8), 
                                  tf.TensorSpec(shape=[None, None, None], dtype=tf.uint8),
                                  tf.TensorSpec(shape=[], dtype=tf.bool),))    
    def _resizing_fn(self, image, mask, resizing_exist):
        # <image> : tf.uint8
        # <mask> : tf.uint8
        if resizing_exist: # branch type : <image> tf.uint8 -> tf.uint8, <mask> tf.uint8 -> tf.uint8
            # Do not remove py_function.
            # does not work because tensorflow make static graph before runtime, tf.function check all kind of cases so it ends up with error.
            image, mask = tf.py_function(func=self._resizing_tf, inp=[image, mask], Tout=[tf.float32, tf.uint8])
            image = tf.cast(image, tf.uint8)
            mask = tf.cast(mask, tf.uint8)
        else:
            pass

        #debugging code
        #tf.print('resizing_fn / <image> val :', tf.math.reduce_min(image), '~', tf.math.reduce_max(image))
        #tf.print('resizing_fn / <image> tensorspec :', tf.TensorSpec.from_tensor(image))

        return image, mask


    def _readmask_tf(self, mask, class_id_to_rgb=None):
        """extract certain classes from mask (e.g. cars)
        """
        if mask.shape[-1] == 1:
            mask = np.squeeze(mask)

        if hasattr(self, 'class_id_to_rgb') and (class_id_to_rgb is None):
            # If overrided class implemented self.class_id_to_rgb
            class_id_to_rgb = self.class_id_to_rgb

        if class_id_to_rgb is None:
            masks = [(mask == v).astype('int') for v in self.class_ids]
        else:
            self.interested_class_id_rgb = {}
            for k, v in class_id_to_rgb.items():
                if k in self.class_ids:
                    self.interested_class_id_rgb[k] = v

            masks = []
            for class_id, colors in self.interested_class_id_rgb.items():
                # e.g. one class has two color {2: [(255, 128, 0), (255, 64, 0)]}
                if len(np.array(colors).shape) == 2:
                    temp_mask = []
                    for color in np.array(colors):
                        temp_mask.append(np.all(mask == color, axis=-1).astype('int'))
                    masks.append(np.logical_or.reduce(temp_mask))
                else:

                    ## debug
                    # print('---\n', colors) ## debug
                    # print(np.all(mask == colors, axis=-1)) ## debug
                    # print('---') ## debug
                    
                    ## debug
                    # plt.figure() ## debug
                    # plt.imshow(np.all(mask == colors, axis=-1)) ## debug

                    masks.append(np.all(mask == colors, axis=-1))

        ## debug
        # print('mask max : {}, min : {}, type : {}'.format(mask.numpy().max(), mask.numpy().min(), mask.numpy().dtype)) ## debug

        # mask = np.stack(masks, axis=-1).astype('int') ## temp
        mask = tf.stack(masks, axis=-1) ## temp
        mask = tf.cast(mask, tf.uint8) ## temp

        # add background if mask is not binary
        if mask.shape[-1] != 1:

            # background = 1 - mask.sum(axis=-1, keepdims=True) ## temp
            background = 1 - tf.math.reduce_sum(mask, axis=-1, keepdims=True) ## temp
            
            ## debug
            # plt.figure() ## debug
            # plt.title('background') ## debug
            # plt.imshow(background.squeeze()) ## debug ## temp
            # plt.imshow(background.numpy().squeeze(), vmax=1, vmin=0) ## debug ## temp
            
            ## debug
            # print('background ({}) max : {}, min : {}, type : {}'.format(background.shape, background.max(), background.min(), background.dtype)) ## debug
            
            # mask = np.concatenate((mask, background), axis=-1) ## temp
            background = tf.cast(background, tf.uint8) ## temp
            mask = tf.concat((mask, background), axis=-1) ## temp

            ## debug
            # print('mask ({}) max : {}, min : {}, type : {}'.format(mask.shape, mask.max(), mask.min(), mask.dtype)) ## debug

            ## debug
            # plt.figure() ## debug
            # plt.title('background-fromtensor') ## debug
            # plt.imshow(mask[:,:,-1], vmax=1, vmin=0) ## debug ## temp
            # plt.imshow(mask[:,:,-1], vmax=1, vmin=0) ## debug ## temp

        ## debug
        # print('mask max : {}, min : {}, type : {}'.format(mask.numpy().max(), mask.numpy().min(), mask.numpy().dtype)) ## debug

        mask = tf.cast(mask, tf.uint8) ## temp
        # mask = mask.astype('uint8') ## temp

        return mask


    def _augmentation_tf(self, image, mask):
        ret = self.augmentation(image=image, mask=mask)
        image = ret['image']
        mask = ret['mask']
        return image, mask


    @tf.function(input_signature=(tf.TensorSpec(shape=[], dtype=tf.string), 
                                  tf.TensorSpec(shape=[], dtype=tf.string),
                                  tf.TensorSpec(shape=[], dtype=tf.bool),))    
    def _read_and_augmentation_fn(self, images_fps, masks_fps, augmentation_exist):
        # <images_fps> : tf.string
        # <masks_fps> : tf.string
        print('Retracing!')

        image = tf.io.read_file(images_fps)
        image = tf.image.decode_png(image, channels=3)
        image = tf.image.convert_image_dtype(image, tf.uint8) 
        # NOTE: Converting integer types to floating point types returns normalized floating point values in the range [0, 1)
        # NOTE: See more, https://www.tensorflow.org/api_docs/python/tf/image/convert_image_dtype

        mask = tf.io.read_file(masks_fps)
        mask = tf.image.decode_png(mask, channels=0) # Select number of channel automatically.
        mask = tf.image.convert_image_dtype(mask, tf.uint8)
        mask = tf.py_function(func=self._readmask_tf, inp=[mask], Tout=tf.uint8)

        if augmentation_exist:
            image, mask = tf.numpy_function(func=self._augmentation_tf, inp=[image, mask], Tout=[tf.uint8, tf.uint8])        

        #debugging code
        #tf.print('read_and_augmentation_fn / <image> val :', tf.math.reduce_min(image), '~', tf.math.reduce_max(image))
        #tf.print('read_and_augmentation_fn / <image> tensorspec :', tf.TensorSpec.from_tensor(image))

        return image, mask


    def get_tf_dataset(self):
        assert self.mode == 'tf.data.Dataset', 'Current mode : {}, If you want to use this method, implement self.images_dataset and self.masks_dataset'.format(self.mode)
        images_dataset = self.images_dataset
        masks_dataset = self.masks_dataset
        self._augmentation_exist = tf.constant(True if self.augmentation is not None else False, dtype=tf.bool, shape=())
        self._resizing_exist = tf.constant(True if self.resizing is not None else False, dtype=tf.bool, shape=())
        self._preprocessing_exist = tf.constant(True if self.preprocessing is not None else False, dtype=tf.bool, shape=())

        tf.print('augmentation :', self._augmentation_exist)
        tf.print('resizing :', self._resizing_exist)
        tf.print('preprocessing :', self._preprocessing_exist)

        n = tf.data.AUTOTUNE
        if n != tf.data.AUTOTUNE:
            print('Warning : parallel call is not AUTOTUNE. It could make pipeline slow.')

        dataset = tf.data.Dataset.zip((images_dataset, masks_dataset)).map(
            lambda im, mask: self._read_and_augmentation_fn(im, mask, self._augmentation_exist), num_parallel_calls=n).map(
                lambda im, mask: self._resizing_fn(im, mask, self._resizing_exist), num_parallel_calls=n).map(
                    lambda im, mask: self._preprocessing_fn(im, mask, self._preprocessing_exist), num_parallel_calls=n)

        return dataset


    def _getitem_keras(self, i):
        # read data
        image = cv2.imread(self.images_fps[i])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # extract certain classes from mask (e.g. cars)
        mask = self._readmask(i)
                
        # apply augmentations
        if self.augmentation:
            sample = self.augmentation(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
        
        # apply final resizing
        if self.resizing:
            image_dt = image.dtype
            mask_dt = mask.dtype
            image, mask = self.resizing(image=image, mask=mask)
            image = image.numpy().astype(image_dt)
            mask = mask.numpy().astype(mask_dt)

        # apply preprocessing
        if self.preprocessing:
            sample = self.preprocessing(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']

        return image, mask


    def __getitem__(self, i): # batch
        if self.mode == 'tf.keras.Sequence':
            return self._getitem_keras(i)
        else:
            print('tf.data.Dataset is not a subscribable object.')
            raise NotImplementedError


    def __len__(self):
        return len(self.images_fps)

def resizing(model_input_w_for_train, model_input_h_for_train):
    def wrapper(image=None, mask=None):
        image = tf.image.resize(image, [model_input_h_for_train, model_input_w_for_train], method='bilinear', antialias=True)
        image = tf.clip_by_value(image, 0.0, 255.0)
        mask = tf.image.resize(mask, [model_input_h_for_train, model_input_w_for_train], method='nearest')
        return image, mask
    return wrapper